In [1]:
import os
import sys
import json

import pandas as pd
import matplotlib.pyplot as plt


import pypsa

data_dir = os.path.abspath("data")
from IPython.display import clear_output

In [2]:
from utils.japan_area_names import cities_jp_to_en
from utils.jp_map_processor import cities_list_en as cities_list

In [3]:
print(pypsa.__version__)

0.26.2


In [4]:
# ============================================================================ #
# load config
# ============================================================================ #

if os.path.exists("config.json"):
	with open("config.json") as config_file:
		config = json.load(config_file)
else:
	config = dict()


# set the RUN_ID. This is used to save the results.
RUN_ID = config.get("RUN_ID")

# set the base folder of the PyPSA Model
pypsa_base_folder = config.get("pypsa_base_folder")

In [5]:
# ============================================================================ #
# Data Preparation
# ============================================================================ #

pypsa_network_folder = os.path.join(data_dir, pypsa_base_folder)
if not os.path.exists(pypsa_network_folder):
    print('Network Folder does not exist.')

# load the power gen
solar_p_max_pu_df = pd.read_csv('data/ref/pv_gen_kwh_per_kw_hourly_main_cities_fy2022.csv', index_col=0)
solar_p_max_pu_df.index = pd.to_datetime(solar_p_max_pu_df.index)

# align the snapshots
solar_p_max_pu_df.reset_index()['index'].to_frame('snapshot').to_csv(f'{pypsa_network_folder}/snapshots.csv')
solar_p_max_pu_df = solar_p_max_pu_df.rename(columns=cities_jp_to_en)

# load the municipal electricity demand
muni_elec_dmd = pd.read_csv("data/gen/major_city_resi_elec_dmd.csv", index_col=0)
muni_elec_dmd.index = pd.to_datetime(muni_elec_dmd.index)
muni_elec_dmd = muni_elec_dmd.rename(columns=cities_jp_to_en)

# load RPV Potential
rpv_capacity_potential = pd.read_csv("data/gen/rpv_capacity_potential.csv", index_col=0)

# load Battery Potential
bat_capacity_potential = pd.read_csv("data/gen/bat_capacity_potential.csv", index_col=0)

In [6]:
# ============================================================================ #
# Scenario Analysis: Model Simulation Execution
# ============================================================================ #

self_suff_rate_df = []
curtailment_rate_df = []
for muni_name in cities_list:
	muni_total_dmd = muni_elec_dmd[muni_name].sum()
	solar_total_output = solar_p_max_pu_df[muni_name].sum()

	# calculate pv capacity levels: 1 - BAU; 2 - new detached; 3 = old and new
	pv_cap_levels = rpv_capacity_potential.loc[muni_name].to_list()

	# calculate battery capacity levels: same as PV levels
	bat_cap_levels = bat_capacity_potential.loc[muni_name].to_list()

	muni_self_suff_rate = []
	muni_curtailment_rate = []
	for has_battery in [False, True]:
		for pv_level in [1, 2, 3]:
			# PyPSA
			# initialize pypsa
			n = pypsa.Network()
			n.import_from_csv_folder(pypsa_network_folder)

			# set the load
			n.loads_t.p_set['residential'] = muni_elec_dmd[muni_name]*1_000

			# set max solar
			n.generators_t.p_max_pu['gen_solar'] = solar_p_max_pu_df[muni_name]

			# set solar capacity
			n.generators.loc['gen_solar', 'p_nom'] = sum(pv_cap_levels[:pv_level])

			# set battery capacity 
			su_name = 'su_battery'
			max_hours = n.storage_units.loc[su_name, 'max_hours']

			if has_battery:
				bat_cap_actual = sum(bat_cap_levels[:pv_level])
			else:
				bat_cap_actual = 0

			n.storage_units.loc[su_name, 'p_nom'] = bat_cap_actual/max_hours

			solver_options = {"NumericFocus": 0, "OutputFlag": 1}
			status, termination_condition = n.optimize(
				solver_name="gurobi",
				solver_options=solver_options,
				# extra_functionality=extra_functionality,
			)

			clear_output(wait=True)

			if status == "ok":
				folder_name = f"{muni_name.lower()}_{int(has_battery)}_{pv_level}"
				save_network_folder = os.path.join(data_dir, 'pypsa-results', RUN_ID, folder_name)
				if not os.path.exists(save_network_folder):
					os.makedirs(save_network_folder)
					print(save_network_folder)

				n.export_to_csv_folder(save_network_folder)




/home/matthew/GitHub/esm-jp-major-cities-rpv-potential/data/pypsa-results/published-ver/tokyo_1_3


INFO:pypsa.io:Exported network tokyo_1_3 has links, buses, storage_units, generators, loads
